##Part 2: Fine-tuning Stable Diffusion

In [ ]:
!nvidia-smi

Thu Nov 30 22:35:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    42W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!cd /content/
!git clone https://github.com/huggingface/diffusers.git
!pip install ./diffusers
!pip install -U -r /content/diffusers/examples/text_to_image/requirements.txt

Cloning into 'diffusers'...
remote: Enumerating objects: 50139, done.
remote: Counting objects: 100% (10953/10953), done.
remote: Compressing objects: 100% (976/976), done.
remote: Total 50139 (delta 10516), reused 10046 (delta 9955), pack-reused 39186
Receiving objects: 100% (50139/50139), 31.14 MiB | 15.93 MiB/s, done.
Resolving deltas: 100% (37222/37222), done.
Processing ./diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.24.0.dev0-py3-none-any.whl size=1764673 sha256=4d89260b1076aeae8cbaf30eda1ab95cee776630d8094a86168f049e9bf86fd3
  Stored in directory: /tmp/pip-ephem-wheel-cache-7dwpmp5h/wheels/95/c5/3b/e1b4269f8a2584de57e75f949a185b48fc4144e9a91fc9965a
Successfully built diffusers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 60.1 MB/s eta 0:0

In [ ]:
!accelerate config default --mixed_precision fp16

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
!pip install torch==2.1.0+cu121 -f https://download.pytorch.org/whl/cu121/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu121/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 814.2 kB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu118
    Uninstalling torch-2.1.0+cu118:
      Successfully uninstalled torch-2.1.0+cu118


In [ ]:
!pip install torchvision==0.16.0+cu121 -f https://download.pytorch.org/whl/cu121/torch_stable.html

Looking in links: https://download.pytorch.org/whl/cu121/torch_stable.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 63.2 MB/s eta 0:00:00
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.16.0+cu118
    Uninstalling torchvision-0.16.0+cu118:
      Successfully uninstalled torchvision-0.16.0+cu118


In [ ]:
import os

os.environ['MODEL_NAME'] = f'CompVis/stable-diffusion-v1-4'
os.environ['DATASET_NAME'] = f'rishthak/albums-mixed'
os.environ['OUTPUT_DIR'] = f'rishthak/album-fine-tune'

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!accelerate config default --mixed_precision fp16

Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


In [ ]:
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 1.7 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.1
    Uninstalling torch-2.1.1:
      Successfully uninstalled torch-2.1.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.16.1 requires torch==2.1.1, but you have torch 2.1.0 which is incompatible.


Run our training script using HuggingFace accelerate. We'll be inputing our dataset and model with 400 steps (selected) and 134 epochs (default). The model will then be automatically pushed to the hub.

In [ ]:
!accelerate launch diffusers/examples/text_to_image/train_text_to_image.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --dataset_name=$DATASET_NAME \
  --use_ema \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --gradient_checkpointing \
  --enable_xformers_memory_efficient_attention \
  --resume_from_checkpoint='checkpoint-900' \
  --mixed_precision="fp16" \
  --max_train_steps=1000 \
  --learning_rate=1e-05 \
  --max_grad_norm=1 \
  --push_to_hub \
  --checkpointing_steps=100 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --output_dir=$OUTPUT_DIR

2023-11-30 23:26:27.942307: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-30 23:26:27.942369: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-30 23:26:27.942394: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-30 23:26:29.213535: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
11/30/2023 23:26:29 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: fp16

{'thresholding', 'prediction_type', 'variance_type', 'sample_max_value'

In [ ]:
!rm -rf rishthak/album-fine-tune/checkpoint-900

In [ ]:
!zip -r album-mixed rishthak

  adding: rishthak/ (stored 0%)
  adding: rishthak/album-fine-tune/ (stored 0%)
  adding: rishthak/album-fine-tune/text_encoder/ (stored 0%)
  adding: rishthak/album-fine-tune/text_encoder/model.safetensors (deflated 8%)
  adding: rishthak/album-fine-tune/text_encoder/config.json (deflated 45%)
  adding: rishthak/album-fine-tune/feature_extractor/ (stored 0%)
  adding: rishthak/album-fine-tune/feature_extractor/preprocessor_config.json (deflated 49%)
  adding: rishthak/album-fine-tune/vae/ (stored 0%)
  adding: rishthak/album-fine-tune/vae/config.json (deflated 56%)
  adding: rishthak/album-fine-tune/vae/diffusion_pytorch_model.safetensors (deflated 8%)
  adding: rishthak/album-fine-tune/checkpoint-1000/ (stored 0%)
  adding: rishthak/album-fine-tune/checkpoint-1000/unet_ema/ (stored 0%)
  adding: rishthak/album-fine-tune/checkpoint-1000/unet_ema/config.json (deflated 65%)
  adding: rishthak/album-fine-tune/checkpoint-1000/unet_ema/diffusion_pytorch_model.safetensors (deflated 7%)
  ad

In [ ]:
# Stable Diffusion V1
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image

model_path = "rap_album_genres_demo"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe.to("cuda")

image1 = pipe(prompt="Album cover for a rap album").images[0]
image1.save("Rap_01.png")

image2 = pipe(prompt="Album cover for a rap album titled 'Liberation'").images[0]
image2.save("Rap_02.png")

image3 = pipe(prompt="Album cover for a rap album titled 'Genesis'").images[0]
image3.save("Rap_03.png")

image4 = pipe(prompt="Album cover for a rap album titled 'Hustle Manifesto'").images[0]
image4.save("Rap_04.png")


# Image grid helper function from HuggingFace
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

all_images = [image1, image2, image3, image4]
grid = image_grid(all_images, rows=1, cols=4)
grid

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_folder(
    folder_path="rishthak/album-fine-tune",
    repo_id="rishthak/album-fine-tune",
)


random_states_0.pkl:   0%|          | 0.00/14.3k [00:00<?, ?B/s]

scaler.pt:   0%|          | 0.00/988 [00:00<?, ?B/s]

optimizer.bin:   0%|          | 0.00/6.88G [00:00<?, ?B/s]

Upload 18 LFS files:   0%|          | 0/18 [00:00<?, ?it/s]

scheduler.bin:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/246M [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

'https://huggingface.co/rishthak/album-fine-tune/tree/main/'

In [ ]:
!pip install accelerate torchvision transformers datasets ftfy tensorboard Jinja2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.10
    Uninstalling wcwidth-0.2.10:
      Successfully uninstalled wcwidth-0.2.10


In [ ]:
from diffusers import StableDiffusionPipeline
import torch
from PIL import Image


pipeline = StableDiffusionPipeline.from_pretrained("rishthak/album-fine-tune")
pipe = pipeline
pipe.to("cuda")

image1 = pipe(prompt="Album cover for a rap album from 2020").images[0]
image1.save("Rap_01.png")

image2 = pipe(prompt="Album cover for a rap album titled 'Liberation' ").images[0]
image2.save("Rap_02.png")

image3 = pipe(prompt="Album cover for a rap album titled 'Genesis'").images[0]
image3.save("Rap_03.png")

image4 = pipe(prompt="Album cover for a rap album titled 'Hustle Manifesto'").images[0]
image4.save("Rap_04.png")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# Image grid helper function from HuggingFace
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size

    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

all_images = [image1, image2, image3, image4]
grid = image_grid(all_images, rows=1, cols=4)
grid

In [ ]:
image1 = pipe(prompt="Album cover for a pop album from 2020").images[0]


image2 = pipe(prompt="Album cover for a indie album titled 'Liberation' ").images[0]


image3 = pipe(prompt="Album cover for a edm album titled 'Genesis'").images[0]


image4 = pipe(prompt="Album cover for a heavy metal album titled 'Hustle Manifesto'").images[0]



all_images = [image1, image2, image3, image4]
grid = image_grid(all_images, rows=1, cols=4)
grid


Output hidden; open in https://colab.research.google.com to view.